# Paper Savior with LionAGI and LlamaIndex Knowledge Graph

-- how to do auto explorative research with LionAGI plus RAG using llamaindex Knowledge Graph Index 

- [LionAGI](https://github.com/lion-agi/lionagi)
- [LlamaIndex](https://www.llamaindex.ai)

install the various dependencies, the first part of this tutorial is based on [Knowledge Graph w/ WikiData Filtering](https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/knowledge_graph2.html)

- if want to read further please check: [Make Meaningful Knowledge Graph from OpenSource REBEL Model](https://medium.com/@haiyangli_38602/make-meaningful-knowledge-graph-from-opensource-rebel-model-6f9729a55527)

The following pip install is for MacOS, if you use cuda GPU, please just `pip install pytorch`, and change the device to `cuda`

In [1]:
# %pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu
# %pip install lionagi llama_index llama_hub transformers wikipedia

In [2]:
query = 'Large Language Model Time Series Analysis'
dir = "data/log/researcher/"
num_papers = 1
num_pages = 3

### 1. Let us write a extriplets extraction function using REBEL model

- [REBEL model](https://huggingface.co/Babelscape/rebel-large) is one of the best models out there for entity extraction. And it's free to use

- we will use wikipedia to filter the entities extracted in order to validate the relationships

In [3]:
from transformers import pipeline

triplet_extractor = pipeline(
    "text2text-generation",
    model="Babelscape/rebel-large",
    tokenizer="Babelscape/rebel-large",
    # comment this line to run on CPU, use "cuda:0" to run on GPU, stay with "mps:0" to run on apple silicon
    device="mps:0",
)

def extract_triplets(input_text):
    text = triplet_extractor.tokenizer.batch_decode(
        [
            triplet_extractor(input_text, return_tensors=True, 
                              return_text=False)[0]["generated_token_ids"]
        ]
    )[0]

    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append(
                    (
                        subject.strip(),
                        relation.strip(),
                        object_.strip()
                    )
                )
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append(
                    (
                        subject.strip(),
                        relation.strip(),
                        object_.strip()
                    )
                )
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token

    if subject != '' and relation != '' and object_ != '':
        triplets.append(
            (
                subject.strip(),
                relation.strip(),
                object_.strip()
            )
        )

    return triplets


import wikipedia

class WikiFilter:
    def __init__(self):
        self.cache = {}

    def filter(self, candidate_entity):
        # check the cache to avoid network calls
        if candidate_entity in self.cache:
            return self.cache[candidate_entity]['title']

        # pull the page from wikipedia -- if it exists
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary,
            }

            # cache the page title and original entity
            self.cache[candidate_entity] = entity_data
            self.cache[page.title] = entity_data

            return entity_data['title']
        except:
            return None


wiki_filter = WikiFilter()

def extract_triplets_wiki(text):
    relations = extract_triplets(text)

    filtered_relations = []
    for relation in relations:
        (subj, rel, obj) = relation
        filtered_subj = wiki_filter.filter(subj)
        filtered_obj = wiki_filter.filter(obj)

        # skip if at least one entity not linked to wiki
        if filtered_subj is None and filtered_obj is None:
            continue

        filtered_relations.append(
            (
                filtered_subj or subj,
                rel,
                filtered_obj or obj,
            )
        )

    return filtered_relations

/Users/lion/anaconda3/envs/lion_dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lion/anaconda3/envs/lion_dev/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/lion/anaconda3/envs/lion_dev/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 2. Build a Knowledge Graph Index with llama_index

In [4]:
from llama_index import KnowledgeGraphIndex, download_loader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.node_parser import SentenceSplitter
from llama_index.graph_stores import SimpleGraphStore
from llama_index.storage.storage_context import StorageContext

ArxivReader = download_loader("ArxivReader")
loader = ArxivReader()
node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=50)

# let us download some papers from arvix
documents, abstracts = loader.load_papers_and_abstracts(search_query=query, max_results=num_papers)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=False)

# set up the knowledge index
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)
llm = OpenAI(temperature=0.1, model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)

index1 = KnowledgeGraphIndex(
    nodes,
    service_context=service_context,
    max_triplets_per_chunk=3,
    kg_triplet_extract_fn=extract_triplets_wiki,
    storage_context=storage_context,
    include_embeddings=True,
)

query_engine = index1.as_query_engine(include_text=False, response_mode="tree_summarize")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/lion/anaconda3/envs/lion_dev/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/lion/anaconda3/envs/lion_dev/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = Bea

In [5]:
storage_context.persist()

In [6]:
len(abstracts)

1

### 3. Write a tool description according to OpenAI schema

In [7]:
import lionagi as li

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_arvix_papers",
            "description": "Perform a query to a QA bot with access to a knowledge graph index built with papers from arvix",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "a question to ask the QA bot",
                    }
                },
                "required": ["query"],
            },
        }
    }
]

func = query_engine.query

In [9]:
system = {
    "persona": "a helpful world-class researcher",
    "requirements": "think step by step before returning a clear, concise, precise worded answer with a humble yet confident tone",
    "responsibilities": f"you are asked to help with researching on the topic of {query}",
    "tools": "provided with a QA bot for grounding responses"
}

### 4. Research: PROMPTS

#### FORMATS

In [10]:
deliver_format1 = {"return required": "yes", "return format": "paragraph"}

deliver_format2 = {"return required": "yes", 
    "return format": {
        "type": "json_mode", 
        "format": {
            'paper': "paper_name",
            "summary": "...", 
            "research question": "...", 
            "talking points": {
                "point 1": "...",
                "point 2": "...",
                "point 3": "..."
            }}}},

deliver_format3 = {
    "notice":"Notice you are provided with a QA bot as your tool, the bot has access to the papers via a queriable knowledge graph index that takes natural language query and return a natural language answer. You can decide whether to invoke the function call, you will need to ask the bot when there are things need clarification or further information. you provide the query by asking a question, please use the tool when appropriately need to", 
    "tool choice": "auto", 
    "format":"function calling"}

deliver_format4 = {
    "return required": "yes", 
    "return format": {
        "type": "json_mode",
        "format": {
            "title": "our new research paper title",
            "summary": "...",
            "key words": ["xxx", "xxx", "xxx"],
            "outline": {
                "part zero: abstract": "plans for abstract",
                "part one: introduction": "...", 
                "part two:...":"...",
                "part three: ...": "...",
                "...": "..."
            }}}}

deliver_format5 = {
    "return required": "yes", 
    "return format": {
        "type": "json_mode",
        "format": {
            "question 1": "...", 
            "question 2": "...",
            "...": "..."
        }}}

#### PROMPTS

In [11]:
instruct1 = {
    "task step": "1", 
    "task name": "read paper abstracts", 
    "task objective": "get initial understanding of the papers of interest", 
    "task description": "provided with abstracts of paper, provide a brief summary highlighting the paper core points",
    "deliverable": deliver_format1
}

instruct2 = {
    "task step": "2",
    "task name": "propose research questions and talking points", 
    "task objective": "initial brainstorming", 
    "task description": "from the improved understanding of the paper, please propose some research questions and talking points.",
    "deliverable": deliver_format2,
    "function calling": deliver_format3
}

instruct3 = {
    "task step": "3",
    "task name": "evaluate and brainstorm new research question",
    "task objective": "identify a most interesting research question and provide talking points",
    "task description": f"provided with a list of research questions and talking points from {num_papers} papers, please thoroughly read through the list and identify the most interesting research question and provide talking points. You need to provide reasoning to support your choice",
    "deliverable": deliver_format1
}

instruct4 = {
    "task step": "4",
    "task name": "solidify the research question",
    "task objective": "create an outline for on research topic",
    "task description": "from our current understanding of task on hand, please propose a final research question as our project",
    "deliverable": deliver_format1,
    "function calling": deliver_format3
}

instruct5 = {
    "task step": "5",
    "task name": "validate outline",
    "task objective": "validate the outline is indeed accurately described to the provided papers",
    "deliverable": deliver_format4, 
    "function calling": deliver_format3
}

instruct6 = {
    "task step": "6",
    "task name": f"write the whole paper draft of around {num_pages} pages, you can do it",
    "deliverable": deliver_format1, 
    "function calling": deliver_format3
}

instruct7 = {
    "task step": "7", 
    "task name": "edit the paper as a whole", 
    "task description": "provided with everything we have worked on, please carefully read through everything and list a series of questions as critique",
    "deliverable": deliver_format1, 
    "function calling": deliver_format3
}

instruct8 = {
    "task step": "8", 
    "task name": "respond to the critism", 
    "task description": "given everything we have talked about, please address the critism and provide a revised version of the paper",
    "deliverable": deliver_format1, 
}

instruct9 = {
    "task step": "9", 
    "task name": "proof reading", 
    "task description": "provided with everything we have talked about, please work on grammar and writing style to sound less robotic, return the whole edited paper, your response limit is 4000 tokens, you can totally deliver the whole thing, take a deep breath, I believe in you",
    "deliverable": deliver_format1, 
}

### 5. Research setup workflow

In [33]:
async def _auto_followup(session, instruct, **kwargs):
    await session.followup(instruct, **kwargs)
    if session.conversation.messages and session.conversation.messages[-1]['role'] == 'tool':
        return await session.followup(instruct, **kwargs)
    return False

async def auto_followup(session, instruct, num_func_call=2, **kwargs):
    for _ in range(num_func_call):
        if not await _auto_followup(session, instruct, **kwargs):
            break
    return session

In [26]:
use_query = {"type":"function", "function":{"name": "query_arvix_papers"}}

In [27]:
# prompt 1-2
async def read_abstract(context):
    researcher = li.Session(system, dir=dir)
    researcher.register_tools(tools, func)
    researcher.llmconfig.update({"tools": tools})
    
    await researcher.initiate(instruct1, context=context, temperature=0.7, tool_choice="none")
    researcher = await auto_followup(researcher, instruct2, temperature=0.4, 
                                     response_format= {'type':'json_object'}, tool_choice=use_query)
    
    researcher.messages_to_csv()
    researcher.log_to_csv()
    return researcher
   

In [28]:
# rest
async def write_paper(context):
    researcher = li.Session(system, dir=dir)
    researcher.register_tools(tools, func)
    researcher.llmconfig.update({"tools": tools})
    
    await researcher.initiate(instruct3, context=context, temperature=0.5, tool_choice="none")
    researcher = await auto_followup(researcher, instruct4, temperature=0.5, tool_choice=use_query)
    researcher = await auto_followup(researcher, instruct5, tempature=0.35, tool_choice=use_query)
    researcher = await auto_followup(researcher, instruct6, temperature=0.75, tool_choice=use_query)
    researcher = await auto_followup(researcher, instruct7, temperature=0.75, tool_choice=use_query)
    await researcher.followup(instruct8, temperature=0.65, tool_choice="none")
    await researcher.followup(instruct9, temperature=0.45, tool_choice="none")

    researcher.messages_to_csv()
    researcher.log_to_csv()
    return researcher.conversation.messages[-1]

### 6. Run the workflow

In [ ]:
# abstracts = li.l_call(range(len(abstracts)), lambda i: abstracts[i].text)

In [34]:
# run first workflow over all paper abstracts
researchers = await li.al_call(abstracts, read_abstract)

5 logs saved to data/log/researcher/_messages_2023-12-14T11_41_06_767212.csv
2 logs saved to data/log/researcher/_llmlog_2023-12-14T11_41_06_769474.csv


In [35]:
researchers[0].conversation.responses

[{'role': 'assistant',
  'content': 'The paper discusses the challenge of time-series analysis across various domains, where typically a unique model is developed for each specific task, requiring substantial data and domain expertise. Addressing this issue, the paper introduces a novel method called Large Pre-trained Time Series Model (LPTM). The key innovation of LPTM is its ability to pre-train on heterogeneous time-series datasets by identifying optimal, task-specific segmentation strategies for the input data, which is accomplished through a self-supervised learning loss. This approach enables the model to handle various time-series dynamics from different domains effectively. The paper claims that LPTM can match or exceed the performance of domain-specific models while being more efficient in terms of data and computational resources. It reportedly requires up to 40% less data and 50% less training time to achieve state-of-the-art results across a diverse range of time-series ana

In [ ]:
# extracts the final outputs from stage 1
out1 = li.l_call(researchers, lambda x: x.conversation.messages[-1])

# run second workflow
paper = await write_paper(out1)

15 logs saved to data/log/researcher/_messages_2023-12-14T11_24_57_780808.csv
7 logs saved to data/log/researcher/_llmlog_2023-12-14T11_24_57_782343.csv


In [38]:
response = func("What are the current limitations of Large Pre-trained Time Series Models (LPTM)?")

In [41]:
response.response

'The limitations of Large Pre-trained Time Series Models (LPTMs) are not specified in the provided context. To understand the current limitations of LPTMs, one would typically need to consult recent research papers, technical reports, or expert analyses that specifically address the performance, challenges, and drawbacks associated with these models.'

In [ ]:
from IPython.display import Markdown

Markdown(paper)

In [ ]:
# let us check the API runs
session = li.Session(system)
session.api_service.status_tracker